In [1]:
from application.database import FirewallRule
from application.database import Citrix
from dataclasses import dataclass
# import urllib3
import urllib
import json
import requests
requests.packages.urllib3.disable_warnings()


Khai Class

In [2]:
class HoTroCntt:
    def __init__(self, url, authtoken) -> None:
        self.url = url
        self.authtoken = authtoken

    def api_get(self, endpoint=None, params=None, data=None):
        try:
            url = f"https://{self.url}/{endpoint}"
            headers = {"authtoken": self.authtoken}
            response = requests.get(
                url, headers=headers, params=params, data=data, verify=False
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return e

    def api_post(self, endpoint, data):
        try:
            url = f"https://{self.url}/{endpoint}"
            headers = {"authtoken": self.authtoken}
            response = requests.post(url, headers=headers, data=data, verify=False)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return e

    def api_delete(self, endpoint, data):
        try:
            url = f"https://{self.url}/{endpoint}"
            headers = {"authtoken": self.authtoken}
            response = requests.delete(url, headers=headers, data=data, verify=False)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return e

    def api_put(self, endpoint, data):
        try:
            url = f"https://{self.url}/{endpoint}"
            headers = {"authtoken": self.authtoken}
            response = requests.put(url, headers=headers, data=data, verify=False)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return e

    def get_requests(self, request_id=None, params=None):
        if request_id is not None:
            return self.api_get(endpoint=f"api/v3/requests/{request_id}")
        return self.api_get(endpoint="api/v3/requests")

    def get_users(self, user_id=None, user_email=None):
        if user_id is not None:
            return self.api_get(endpoint=f"api/v3/users/{user_id}")
        if user_email is not None:
            input_data = json.dumps(
                {
                    "list_info": {
                        "get_total_count": True,
                        "search_fields": {"email_id": user_email},
                    }
                }
            )
            params = {"input_data": input_data}
            return self.api_get(endpoint="api/v3/users", params=params)
        return self.api_get(endpoint="api/v3/users")

    def get_sites(self, site_id=None, params=None):
        if site_id is not None:
            return self.api_get(endpoint=f"api/v3/sites/{site_id}")
        return self.api_get(endpoint="api/v3/sites")

    def get_technicians(self, technician_id=None, params=None):
        if technician_id is not None:
            return self.api_get(endpoint=f"api/v3/technicians/{technician_id}")
        return self.api_get(endpoint="api/v3/technicians")

    def get_categories(self, category_id=None, params=None):
        if category_id is not None:
            return self.api_get(endpoint=f"/api/v3/categories/{category_id}")
        return self.api_get(endpoint="api/v3/categories")

    def get_request_approval_levels(self, request_id, approval_level_id=None):
        if approval_level_id is not None:
            return self.api_get(
                endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}"
            )
        return self.api_get(endpoint=f"api/v3/requests/{request_id}/approval_levels")

    def delete_request_approval_levels(self, request_id, approval_level_id):
        return self.api_get(
            endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}"
        )

    def add_request_approval(self, request_id, approval_level_id, approver_id):
        input_data = json.dumps({"approval": {"approver": {"id": approver_id}}})
        payload = {"input_data": input_data}
        return self.api_post(
            endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}/approvals",
            data=payload,
        )

    def delete_request_approval(self, request_id, approval_level_id, approval_id):
        input_data = {}
        payload = {"input_data": input_data}
        return self.api_delete(
            endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}/approvals/{approval_id}",
            data=payload,
        )

    def send_notification_for_request_approval(
        self, request_id, approval_level_id, approval_id
    ):
        input_data = json.dumps(
            {
                "notification": {
                    "subject": "Approval required for a Request",
                    "description": "Your approval is required for a Request to act upon. The details of the Request can be found at $ApprovalLink",
                }
            }
        )
        payload = {"input_data": input_data}
        return self.api_put(
            endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}/approvals/{approval_id}/_send_notification",
            data=payload,
        )

    def get_request_approvals(self, request_id, approval_level_id, approval_id=None):
        if approval_id is not None:
            return self.api_get(
                endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}/approvals/{approval_id}"
            )
        return self.api_get(
            endpoint=f"api/v3/requests/{request_id}/approval_levels/{approval_level_id}/approvals"
        )

    def submit_for_approval(self, request_id, approver_id):
        try:
            request_details = self.get_requests(request_id=request_id, params=None)
            request_subject = request_details.get("request").get("subject")
            requester_name = request_details.get("request").get("requester").get("name")
            requester_email = (
                request_details.get("request").get("requester").get("email_id")
            )
            requester_mobile = (
                request_details.get("request").get("requester").get("mobile")
            )
            request_template = (
                request_details.get("request").get("udf_fields").get("udf_sline_3019")
            )
            notification_title = (
                f"Yêu cầu chờ phê duyệt {request_id} từ hotrocntt.cpc.vn"
            )
            notification_description = f"""<div>
                    Bạn có một yêu cầu phê duyệt với nội dung như sau:<br>
                    - Người yêu cầu: {requester_name}<br>
                    - Email: {requester_email}<br>
                    - Mobile: {requester_mobile}<br><br>
                    Nội dung yêu cầu:<br>
                    - Phê duyệt khởi tạo cấu hình {request_template.upper()}<br>
                    - Tiêu đề: {request_subject} <br>
                    - Nội dung cấu hình xem tại: <a target="_blank" href="https://nssa.dc.vn/servicedesk/citrix/{request_id}">https://nssa.cpc.vn/servicedesk/{request_template}/{request_id}</a><br><br>
                    
                    Anh/Chị vui lòng truy cập link sau để xem chi tiết và thực hiện phê duyệt: <a target="_blank" href="$ApprovalLink">Phê Duyệt</a>
                </div>"""
            input_data = {
                "approvals": [
                    {"approver": {"id": approver}} for approver in approver_id
                ],
                "notification": {
                    "title": notification_title,
                    "description": notification_description,
                },
            }
            payload = "input_data=" + urllib.parse.quote(json.dumps(input_data))
            headers = {
                "Accept": "vnd.manageengine.v3+json",
                "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                "authtoken": "BB46BAAD-8F3B-47CA-900B-A4B83CA50314",
            }
            url = f"https://{self.url}/api/v3/requests/{request_id}/submit_for_approval"
            response = requests.request("POST", url, headers=headers, data=payload)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return e

    def get_request_approval_status(self, request_id) -> dict:
        request_details = self.get_requests(request_id=request_id)
        if request_details.get("request"):
            return request_details.get("request").get("approval_status")
        return {"code": 0, "message": request_details}
    
    def get_request_approver_list(self,request_id) -> dict:
        has_more_rows = True
        approver = []
        while has_more_rows is True:
            result = self.api_get(endpoint=f"api/v3/requests/{request_id}/approval_levels/approvals/approver")
            if result.get("response_status").get("status_code") == 2000 and result.get("response_status").get("status") == "success":    
                approver.extend(result.get("approver"))
                has_more_rows = result.get("list_info").get("has_more_rows")
            else:
                return {"code": 0, "message": "error"}
        return approver

Khai API

In [7]:
api_hotrocntt = HoTroCntt(
    url="hotrocntt.cpc.vn", authtoken="564C437D-62C0-4458-8B16-53847F2EEAD9"
)

mysql_citrix = Citrix()

In [ ]:
approver_list = api_hotrocntt.get_request_approver_list(request_id="108036")

Khai request_id

In [12]:
request_id = "1080401"
servicedesk_request = mysql_citrix.get_servicedesk_citrix_request(
        requestid=request_id
    )
if type(servicedesk_request) is not (dict or json):
    print(False)

In [200]:
api_hotrocntt.submit_for_approval(request_id=request_id,approver_id=["52834","22503"])

{'response_status': {'status_code': 2000, 'status': 'success'},
 'submit_for_approval': {'approvals': [{'approver': {'email_id': 'TuanPA2@cpc.vn',
     'phone': '0906307547',
     'name': 'Phan Anh Tuấn (IT-KTM.CV)',
     'mobile': '0906307547',
     'profile_pic': {'content-url': '/images/default-profile-pic2.svg',
      'name': 'default-profile-pic2.svg'},
     'is_vipuser': False,
     'id': '52834',
     'department': {'site': {'name': 'CPCIT', 'id': 315},
      'name': 'Phòng Kỹ thuật Mạng viễn thông',
      'id': 7225}},
    'comments': None,
    'approval_level': {'request': {'id': '108040'},
     'level': 1,
     'purchase_order': None,
     'release': None,
     'change': None,
     'name': 'Level One',
     'id': '6926',
     'status': {'id': '1'}},
    'sent_on': {'display_value': 'May 31, 2023 10:50 PM',
     'value': '1685548214017'},
    'org_role': None,
    'action_by': None,
    'deleted': False,
    'sent_by': {'email_id': 'TuanPA2@cpc.vn',
     'phone': '0906307547',

In [14]:
class response:
    success: bool
    message: str

a = response
a.success = False
a.message = "aldskjfdf"

if a.success is False:
    print(a.message)


aldskjfdf


Lấy thông tin request

In [186]:
ticket = api_hotrocntt.get_request_approval_status(request_id="107893")
print(ticket)

{'name': 'Approved', 'id': '2'}


Lấy danh sách approval levels

In [181]:
approval_levels = api_hotrocntt.get_request_approval_levels(request_id=request_id)
print(json.dumps(approval_levels))
approval_level_id = []
for element in approval_levels.get("approval_levels"):
    approval_level_id.append(element.get("id"))
print(approval_level_id)

{"response_status": [{"status_code": 2000, "status": "success"}], "approval_levels": [{"request": {"subject": "test citrix 4", "id": "108036"}, "comments": null, "level": 1, "associated_entity": "request", "release": null, "action_taken_by": {"email_id": "TuanPA2@cpc.vn", "phone": "0906307547", "name": "Phan Anh Tu\u1ea5n (IT-KTM.CV)", "mobile": "0906307547", "profile_pic": {"content-url": "/images/default-profile-pic2.svg", "name": "default-profile-pic2.svg"}, "is_vipuser": false, "id": "52834", "department": {"site": {"name": "CPCIT", "id": 315}, "name": "Ph\u00f2ng K\u1ef9 thu\u1eadt M\u1ea1ng vi\u1ec5n th\u00f4ng", "id": 7225}}, "change": null, "rule": {"type": "template_configurations", "value": "all"}, "created_by": {"email_id": "TuanPA2@cpc.vn", "phone": "0906307547", "name": "Phan Anh Tu\u1ea5n (IT-KTM.CV)", "mobile": "0906307547", "profile_pic": {"content-url": "/images/default-profile-pic2.svg", "name": "default-profile-pic2.svg"}, "is_vipuser": false, "id": "52834", "departm

Tạo approval

In [54]:
add_approval = api_hotrocntt.add_request_approval(request_id=request_id,approval_level_id=6913,approver_id=52834)
print(add_approval)

404 Client Error:  for url: https://hotrocntt.cpc.vn/api/v3/requests/107986/approval_levels/6913/approvals


Lấy approval_id

In [ ]:
print(add_approval.get("approval").get("id"))

In [26]:
send_approval = api_hotrocntt.send_notification_for_request_approval(request_id='107904',approval_level_id='6913',approval_id='30701')
print(send_approval)

404 Client Error:  for url: https://hotrocntt.cpc.vn/api/v3/requests/107904/approval_levels/6913/approvals/30701/send_notification


Lấy thông tin người dùng theo email:

In [141]:
users = api_hotrocntt.get_users(user_email="lanhtv@cpc.vn")
for user in users.get("users"):
    a = {
        "email_id": user.get("email_id"),
        "id": user.get("id"),
        "is_technician": user.get("is_technician"),
        "login_name": user.get("login_name"),
        "name": f"{user.get('last_name')} {user.get('first_name')}"
    }
    print(a)

{'email_id': 'lanhtv@cpc.vn', 'id': '22503', 'is_technician': True, 'login_name': 'lanhtv', 'name': 'Trần Văn Lanh'}


Lấy danh sách approvals

In [5]:
approval = api_hotrocntt.get_request_approvals(request_id=request_id,approval_level_id=6913)
with open("approvals.json", mode="w") as f:
    f.write(json.dumps(approval,indent=4))
approval_id = []
for element in approval.get("approvals"):
    approval_id.append(element.get("id"))
print(approval_id)

['30697', '30698', '30699']


In [6]:
final_data = []
keys = ["id","status","deleted"]
for item in approval_id:
    a = {}
    for key in keys:
        a[key] =  api_hotrocntt.get_request_approvals(request_id=107904,approval_level_id=6913,approval_id=item).get("approval").get(key)
    final_data.append(a)
print(final_data)

[{'id': '30697', 'status': {'name': 'Approved', 'id': '2'}, 'deleted': False}, {'id': '30698', 'status': {'name': 'Pending Approval', 'id': '1'}, 'deleted': False}, {'id': '30699', 'status': {'name': 'Pending Approval', 'id': '1'}, 'deleted': True}]


Xóa request